In [ ]:
import import_ipynb
import pygame
from settings import *
from support import *
from timer import Timer
from monster import Monster, Opponent
from random import choice
from random import randint
from ui import UI

class Game:
    def __init__(self):
        pygame.init()
        self.display_surface = pygame.display.set_mode((WINDOW_WIDTH, WINDOW_HEIGHT))
        pygame.display.set_caption('Monster Battle')
        main_font = pygame.font.SysFont("Bungee Regular", 50)
        self.clock = pygame.time.Clock()
        self.running = True
        # using the dictionary made from import_assets()
        self.import_assets()
        self.player_active = True

        # useless
        # groups 
        self.all_sprites = pygame.sprite.Group()

        # useless
        # data; imports from monster.ipyn; make sure names in list alligns with the file names u r importing
        player_monster_list = ['Sparchu', 'Cleaf', 'Jacana']

        # useless
        # player
        self.player_monsters = [Monster(name, self.back_surfs[name]) for name in player_monster_list]
        self.monster = self.player_monsters[randint(0,2)]
        self.all_sprites.add(self.monster)
        
        # useless
        # opponent
        opponent_name = choice(list(MONSTER_DATA.keys()))
        self.opponent_monsters = Opponent(opponent_name, self.front_surfs[opponent_name],self.all_sprites)

        # alive & broken status
        self.player4_alive = True
        self.player1_alive = True
        self.player2_alive = True
        self.player3_alive = True
        
        self.player4_broken = False
        self.player1_broken = False
        self.player2_broken = False 
        self.player3_broken = False
        
        # list of moves made against a player
        self.player4_def = []
        self.player1_def = []
        self.player2_def = []
        self.player3_def = []        
        
        # ui
        self.ui = UI(self.monster, self.get_input)
        self.counter = 0

        #timers
        self.timers = {'player1 end': Timer(10, func = self.player2_turn),
                      'player2 end': Timer(10, func = self.player3_turn),
                      'player3 end': Timer(10, func = self.player4_turn),
                       'player4 end': Timer(10, func = self.end_turn),
                      'end turn': Timer(10000, func = self.player1_turn)}

    # when we get input in ui.ipynb, we get values in main.ipynb
    def get_input(self, state, data):
        if self.counter == 0:
            self.counter += 1
            self.timers['player1 end'].activate()
        elif self.counter == 1:
            self.counter += 1
            self.timers['player2 end'].activate()           
        elif self.counter == 2:
            self.counter += 1  
            self.timers['player3 end'].activate()                 
        elif self.counter == 3:
            self.counter += 1 
            self.timers['player4 end'].activate()
            self.counter = 0
            
        print(state)
        print(data)

    # getting the effect of interaction
    def apply_move(self, player4_move, player1_move, player2_move, player3_move):

        # addition for charges
        if player4_move == 'Charge':
            self.ui.player4_charges += 1
        if player1_move == 'Charge':
            self.ui.player1_charges += 1
        if player2_move == 'Charge':
            self.ui.player2_charges += 1
        if player3_move == 'Charge':
            self.ui.player3_charges += 1

        # subtraction for charges
        try:
            self.ui.player4_charges -= COST_DATA[player4_move]
        except:
            self.ui.player4_charges -= 0
        try:
            self.ui.player1_charges -= COST_DATA[player1_move]
        except:
            self.ui.player1_charges -= 0
        try:
            self.ui.player2_charges -= COST_DATA[player2_move]
        except:
            self.ui.player2_charges -= 0
        try:
            self.ui.player3_charges -= COST_DATA[player3_move]
        except:
            self.ui.player3_charges -= 0
            

        # effect
        if player1_move == 'Repair':
            self.player1_broken = False
        if player1_move in ['Shield', 'Warshield', 'Aura']:
            if self.player1_broken == True:
                player1_move = 'Teleport'  
                print('player 1 broken; forced to teleport')
        try:
            player_opp1_effect = INTERACTION_DATA[player4_move][player1_move]
        except:
            player_opp1_effect = '-'  
        try:
            opp2_opp1_effect = INTERACTION_DATA[player2_move][player1_move]
        except:
            opp2_opp1_effect = '-'  
        try:
            opp3_opp1_effect = INTERACTION_DATA[player3_move][player1_move]
        except:
            opp3_opp1_effect = '-'  

        if player2_move == 'Repair':
            self.player2_broken = False
        if player2_move in ['Shield', 'Warshield', 'Aura']:
            if self.player2_broken == True:
                player2_move = 'Teleport'  
                print('player 2 broken; forced to teleport')
        try:
            player_opp2_effect = INTERACTION_DATA[player4_move][player2_move]
        except:
            player_opp2_effect = '-'  
        try:
            opp1_opp2_effect = INTERACTION_DATA[player1_move][player2_move]
        except:
            opp1_opp2_effect = '-'  
        try:
            opp3_opp2_effect = INTERACTION_DATA[player3_move][player2_move]
        except:
            opp3_opp2_effect = '-' 

        if player3_move == 'Repair':
            self.player3_broken = False        
        if player3_move in ['Shield', 'Warshield', 'Aura']:
            if self.player3_broken == True:
                player3_move = 'Teleport'  
                print('player 3 broken; forced to teleport')
        try:
            player_opp3_effect = INTERACTION_DATA[player4_move][player3_move]
        except:
            player_opp3_effect = '-'  
        try:
            opp1_opp3_effect = INTERACTION_DATA[player1_move][player3_move]
        except:
            opp1_opp3_effect = '-'  
        try:
            opp2_opp3_effect = INTERACTION_DATA[player2_move][player3_move]
        except:
            opp2_opp3_effect = '-' 
        

        if player4_move == 'Repair':
            self.player4_broken = False
        if player4_move in ['Shield', 'Warshield', 'Aura']:
            if self.player4_broken == True:
                player4_move = 'Teleport' 
                print('player broken; forced to teleport')
        try:
            opp1_player_effect = INTERACTION_DATA[player1_move][player4_move]
        except:
            opp1_player_effect = '-'  
        try:
            opp2_player_effect = INTERACTION_DATA[player2_move][player4_move]
        except:
            opp2_player_effect = '-'  
        try:
            opp3_player_effect = INTERACTION_DATA[player3_move][player4_move]
        except:
            opp3_player_effect = '-'        

        # check for overload discharge
        if self.ui.player1_charges < 0:
            self.player1_alive = False
            opp1_player_effect = '-'  
            opp1_opp2_effect = '-' 
            opp1_opp3_effect = '-' 
        if self.ui.player2_charges < 0:
            self.player2_alive = False
            opp2_player_effect = '-'  
            opp2_opp1_effect = '-' 
            opp2_opp3_effect = '-' 
        if self.ui.player3_charges < 0:
            self.player3_alive = False
            opp3_player_effect = '-'  
            opp3_opp1_effect = '-' 
            opp3_opp2_effect = '-' 
        if self.ui.player4_charges < 0:
            self.player4_alive = False
            player_opp1_effect = '-'  
            player_opp2_effect = '-' 
            player_opp3_effect = '-' 

        # compiling effects to said player
        if self.ui.targets[0] in ['AOE', 'player 1', 'Not player 2', 'Not player 3']:
            self.player1_def.append(player_opp1_effect)
        if self.ui.targets[2] in ['AOE', 'player 1', 'Not player 3', 'Not player4']:
            self.player1_def.append(opp2_opp1_effect)
        if self.ui.targets[3] in ['AOE', 'player 1', 'Not player 2', 'Not player4']:
            self.player1_def.append(opp3_opp1_effect)        
        
        if self.ui.targets[0] in ['AOE', 'player 2', 'Not player 1', 'Not player 3']:
            self.player2_def.append(player_opp2_effect)
        if self.ui.targets[1] in ['AOE', 'player 2', 'Not player 3', 'Not player4']:
            self.player2_def.append(opp1_opp2_effect)
        if self.ui.targets[3] in ['AOE', 'player 2', 'Not player 1', 'Not player4']:
            self.player2_def.append(opp3_opp2_effect)

        if self.ui.targets[0] in ['AOE', 'player 3', 'Not player 1', 'Not player 2']:
            self.player3_def.append(player_opp3_effect)
        if self.ui.targets[1] in ['AOE', 'player 3', 'Not player 2', 'Not player4']:            
            self.player3_def.append(opp1_opp3_effect)
        if self.ui.targets[2] in ['AOE', 'player 3', 'Not player 1', 'Not player4']:
            self.player3_def.append(opp2_opp3_effect)
        
        if self.ui.targets[1] in ['AOE', 'player4', 'Not player 2', 'Not player 3']:        
            self.player4_def.append(opp1_player_effect)
        if self.ui.targets[2] in ['AOE', 'player4', 'Not player 1', 'Not player 3']:           
            self.player4_def.append(opp2_player_effect)
        if self.ui.targets[3] in ['AOE', 'player4', 'Not player 1', 'Not player 2']:   
            self.player4_def.append(opp3_player_effect)

        print(self.player4_def, self.player1_def, self.player2_def, self.player3_def)
        
        # kill;break;aura proc;nothing;teleport proc
        if 'Kill' in self.player1_def:
            self.player1_alive = False
        if 'Breaks' in self.player1_def:
            self.player1_broken = True
            print('player1 Broke')
        if 'Gain 1 Ch' in self.player1_def:
            for i in range(len(self.player1_def)):
                if self.player1_def[i] == 'Gain 1 Ch':
                    self.ui.player1_charges += 1
                    print('Aura Proc: player1 gained 1 Ch')
        if player1_move == 'Teleport':
            for i in range(len(self.player1_def)):
                try:
                    if randint(1,8) <= (8*self.player1_def[i]):
                        print('player1 Teleport Successful')
                    else:
                        print('player1 Teleport Failed')
                        self.player1_alive = False
                except:
                    1==1

        if 'Kill' in self.player2_def:
            self.player2_alive = False
        if 'Breaks' in self.player2_def:
            self.player2_broken = True
            print('player2 Broke')
        if 'Gain 1 Ch' in self.player2_def:
            for i in range(len(self.player2_def)):
                if self.player2_def[i] == 'Gain 1 Ch':
                    self.ui.player2_charges += 1
                    print('Aura Proc: player2 gained 1 Ch')
        if player2_move == 'Teleport':
            for i in range(len(self.player2_def)):
                try:
                    if randint(1,8) <= (8*self.player2_def[i]):
                        print('player2 Teleport Successful')
                    else:
                        print('player2 Teleport Failed')
                        self.player2_alive = False
                except:
                    1==1

        if 'Kill' in self.player3_def:
            self.player3_alive = False
        if 'Breaks' in self.player3_def:
            self.player3_broken = True
            print('player3 Broke')
        if 'Gain 1 Ch' in self.player3_def:
            for i in range(len(self.player3_def)):
                if self.player3_def[i] == 'Gain 1 Ch':
                    self.ui.player3_charges += 1
                    print('Aura Proc: player3 gained 1 Ch')
        if player3_move == 'Teleport':
            for i in range(len(self.player3_def)):
                try:
                    if randint(1,8) <= (8*self.player3_def[i]):
                        print('player3 Teleport Successful')
                    else:
                        print('player3 Teleport Failed')
                        self.player3_alive = False
                except:
                    1==1

        if 'Kill' in self.player4_def:
            self.player4_alive = False
        if 'Breaks' in self.player4_def:
            self.player4_broken = True
            print('Player4 Broke')
        if 'Gain 1 Ch' in self.player4_def:
            for i in range(len(self.player4_def)):
                if self.player4_def[i] == 'Gain 1 Ch':
                    self.ui.player4_charges += 1
                    print('Aura Proc: Player4 gained 1 Ch')
        if player4_move == 'Teleport':
            for i in range(len(self.player4_def)):
                try:
                    if randint(1,8) <= (8*self.player4_def[i]):
                        print('Player4 Teleport Successful')
                    else:
                        print('Player4 Teleport Failed')
                        self.player4_alive = False
                except:
                    1==1
        
        self.kill()
        
        # charge count display
        print('player4 has '+str(self.ui.player4_charges)+' Charge/s')
        print('player1 has '+str(self.ui.player1_charges)+' Charge/s')
        print('player2 has '+str(self.ui.player2_charges)+' Charge/s')
        print('player3 has '+str(self.ui.player3_charges)+' Charge/s')

        # winner display
        if self.player4_alive == False and self.player1_alive == False and self.player2_alive == False and self.player3_alive == False:
            print('No one Wins')
        elif self.player1_alive == False and self.player2_alive == False and self.player3_alive == False:
            print('Player4 Wins')
        elif self.player4_alive == False and self.player2_alive == False and self.player3_alive == False:
            print('player 1 Wins')
        elif self.player4_alive == False and self.player1_alive == False and self.player3_alive == False:
            print('player 2 Wins')            
        elif self.player4_alive == False and self.player1_alive == False and self.player2_alive == False:
            print('player 3 Wins')

    
        self.ui.turn_moves = []
        self.player4_def = []
        self.player1_def = []
        self.player2_def = []
        self.player3_def = []
        self.ui.targets = []
        

    # kill command
    def kill(self):
        if self.player4_alive == False:
            print('player death sequence')
            self.ui.player4_charges -= 100
        if self.player1_alive == False:
            print('player1 death sequence')
            self.ui.player1_charges -= 100
        if self.player2_alive == False:
            print('player2 death sequence')
            self.ui.player2_charges -= 100
        if self.player3_alive == False:
            print('player3 death sequence')
            self.ui.player3_charges -= 100
            
    
    # opponent123 all just do random attacking moves
    # def opponent1_turn(self):
    #     OpponentATKCh = choice(self.ui.attacks)
    #     Attack = choice(ATTACK_DATA[OpponentATKCh])
    #     #experimental
    #     self.ui.turn_moves.append(Attack)
    #     print(Attack)
    #     if TARGET_DATA[Attack] == 'Single':
    #         self.ui.targets.append(choice(self.ui.single_target)) 
    #     elif TARGET_DATA[Attack] == 'Double':
    #         self.ui.targets.append(choice(self.ui.double_target)) 
    #     else:
    #         self.ui.targets.append('AOE')              
            
    #     self.timers['opponent1 end'].activate()

    # def opponent2_turn(self):
    #     OpponentATKCh = choice(self.ui.attacks)
    #     Attack = choice(ATTACK_DATA[OpponentATKCh])
    #     #experimental
    #     self.ui.turn_moves.append(Attack)
    #     print(Attack)
    #     if TARGET_DATA[Attack] == 'Single':
    #         self.ui.targets.append(choice(self.ui.single_target)) 
    #     elif TARGET_DATA[Attack] == 'Double':
    #         self.ui.targets.append(choice(self.ui.double_target)) 
    #     else:
    #         self.ui.targets.append('AOE')  

        # self.timers['opponent2 end'].activate()

    # def opponent3_turn(self):
    #     OpponentATKCh = choice(self.ui.attacks)
    #     Attack = choice(ATTACK_DATA[OpponentATKCh])
    #     #experimental
    #     self.ui.turn_moves.append(Attack)
    #     print(Attack)
    #     if TARGET_DATA[Attack] == 'Single':
    #         self.ui.targets.append(choice(self.ui.single_target)) 
    #     elif TARGET_DATA[Attack] == 'Double':
    #         self.ui.targets.append(choice(self.ui.double_target)) 
    #     else:
    #         self.ui.targets.append('AOE')  
        
    #     self.timers['opponent3 end'].activate()

    #     print(self.ui.targets)
    #     print(self.ui.turn_moves)
    #     print(self.ui.turn_moves[0],self.ui.turn_moves[1],self.ui.turn_moves[2],self.ui.turn_moves[3])
    #     self.apply_move(self.ui.turn_moves[0], self.ui.turn_moves[1],self.ui.turn_moves[2],self.ui.turn_moves[3])

    
    def player1_turn(self):
        self.player_active = True

    def player2_turn(self):
        self.player_active = True    

    def player3_turn(self):
        self.player_active = True    

    def player4_turn(self):
        self.player_active = True
        
    def end_turn(self):
        self.player_active = False
        print(self.ui.targets)
        print(self.ui.turn_moves)
        print(self.ui.turn_moves[0],self.ui.turn_moves[1],self.ui.turn_moves[2],self.ui.turn_moves[3])
        self.apply_move(self.ui.turn_moves[0], self.ui.turn_moves[1],self.ui.turn_moves[2],self.ui.turn_moves[3])
        self.timers['end turn'].activate()
    
    def update_timers(self):
        for timer in self.timers.values():
            timer.update()

    # useless
    # importing images from folders
    def import_assets(self):
        self.back_surfs = folder_importer('images', 'back')
        self.bg_surfs = folder_importer('images', 'other')
        self.front_surfs = folder_importer('images', 'front')

    # useless
    def draw_monster_floor(self):
        for sprite in self.all_sprites:
            floor_rect = self.bg_surfs['floor'].get_frect(center = sprite.rect.midbottom + pygame.Vector2(0, -10))
            self.display_surface.blit(self.bg_surfs['floor'], floor_rect)

    def run(self):
        while self.running:
            dt = self.clock.tick() / 1000
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False
           
            # update (remove 'if self.player_active:' for more freedom)
            self.update_timers()
            self.all_sprites.update(dt)
            if self.player_active:
                self.ui.update()

            # useless
            # draw  
            self.display_surface.blit(self.bg_surfs['bg'],(0,0))
            self.draw_monster_floor()
            self.all_sprites.draw(self.display_surface)
            self.ui.draw()
            pygame.display.update()
        
        pygame.quit()
    
if __name__ == '__main__':
    game = Game()
    
    game.run() 

pygame-ce 2.5.0 (SDL 2.30.3, Python 3.11.7)
importing Jupyter notebook from settings.ipynb
importing Jupyter notebook from support.ipynb
importing Jupyter notebook from timer.ipynb
importing Jupyter notebook from monster.ipynb
importing Jupyter notebook from ui.ipynb


/opt/anaconda3/lib/python3.11/site-packages/pygame/sysfont.py:490: UserWarning: The system font 'Bungee Regular' couldn't be found. Using the default font instead.
  warnings.warn(
